In [1]:
import hoomd
import gsd.hoomd
import numpy
import math
import itertools 

In [2]:
import os

fn = os.path.join(os.getcwd(), 'dimer_centers.gsd')
![ -e "$fn" ] && rm "$fn"

In [3]:
#dimer_positions = [[-0.5, 0, 0], [0.5, 0, 0]]

In [4]:
central_rotation = 0.9
central_position = [2, 1, 0]

In [5]:
s = gsd.hoomd.Snapshot()
s.particles.N = 2
s.particles.types = ['A']
s.particles.typeid = [0,0]
s.particles.position = [[1,1,1], [2,1,2]]
s.particles.orientation = [[1, 0, 0, 0], [0, 0.70710678118, 0.0, 0.70710678118]]
s.configuration.box = [8, 8, 8, 0, 0, 0]
s.particles.mass = [2] * 2

/Users/noah/miniconda3/envs/try2/lib/python3.10/site-packages/gsd/hoomd.py:420: FutureWarning: Snapshot is deprecated, use Frame
  warnings.warn("Snapshot is deprecated, use Frame", FutureWarning)


In [6]:
#Script in order to account for Intertia, which hopefully solves the line 9 problem of providing no torques
#The problem: adding the mass property to the particles breaks the line 4 command
#general_positions = numpy.array([[0, 0, 0], [1, 0, 0]])

#I_ref = numpy.array([[0.4, 0, 0],
#                   [0, 0.4, 0],
#                   [0, 0, 0.4]])
#I_general = numpy.zeros(shape=(3,3))
#for r in general_positions:
#    I_general += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
    
#I_diagonal, E_vec = numpy.linalg.eig(I_general)

#R = E_vec.T

#diagonal_positions = numpy.dot(R, general_positions.T).T

#I_check = numpy.zeros(shape=(3,3))
#for r in diagonal_positions:
#    I_check += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))

mass = 1
I = numpy.zeros(shape=(3,3))
for r in central_position:
    I += mass * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
I

array([[ 0., -5., -5.],
       [-5.,  0., -5.],
       [-5., -5.,  0.]])

In [7]:
s.particles.moment_intertia = [I[0, 0], I[1, 1], I[2, 2]] * 2

In [8]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
sim.create_state_from_snapshot(s)

In [9]:
rigid = hoomd.md.constrain.Rigid()

#rigid.body['dimer'] = {
#    "constituent_types": ['A', 'A'],
#    "positions": [[-0.5, 0, 0], [0.5, 0, 0]],
#    "orientations": [(1.0, 0.0, 0.0, 0.0), (1.0, 0, 0, 1.0)],
#    "charges": [0.0, 0.0],
#    "diameters": [1.0, 1.0]
#}

In [10]:
integrator = hoomd.md.Integrator(dt=0.005, integrate_rotational_dof=True)

cell = hoomd.md.nlist.Cell(buffer=0.4)

gay_berne = hoomd.md.pair.aniso.GayBerne(nlist=cell, default_r_cut=2.5)
gay_berne.params[('A', 'A')] = dict(epsilon=1.0, lperp=0.2, lpar=0.7)
gay_berne.r_cut[('A', 'A')] = 2.5

In [11]:
sim.operations.integrator = integrator

In [12]:
integrator.rigid = rigid

In [13]:
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

kT = 1.5
rigid_centers_and_free = hoomd.filter.Rigid(("center", "free"))
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
integrator.forces.append(gay_berne)

/Users/noah/miniconda3/envs/try2/lib/python3.10/site-packages/hoomd/md/methods/methods.py:102: FutureWarning: NVT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantVolume method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


In [14]:
sim.state.thermalize_particle_momenta(filter=rigid_centers_and_free, kT=kT)
sim.run(0)
nvt.thermalize_thermostat_dof()

In [15]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
gsd_writer = hoomd.write.GSD(filename = 'traj.gsd', trigger = hoomd.trigger.Periodic(10), mode = 'wb', filter = hoomd.filter.All())
sim.operations.writers.append(gsd_writer)

In [16]:
sim.operations.computes.append(thermodynamic_properties)
sim.run(5000)

In [17]:
g = sim.state.get_snapshot()

In [18]:
g.particles.orientation

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.70710678, 0.        , 0.70710678]])

In [19]:
thermodynamic_quantities = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_quantities)

In [20]:
translational_degrees_of_freedom = thermodynamic_quantities.translational_degrees_of_freedom
print('Translational degrees of freedon:', translational_degrees_of_freedom)

Translational degrees of freedon: 6.0


In [21]:
thermodynamic_quantities.rotational_kinetic_energy

0.0

In [22]:
#Current State: Everything "works" The only problem is that the particles are not rotating, and don't have any
# rotational kinetic energy, the source of the problem is likely found 06 01 which provides the initial system,
# which I believe provides the rotational kinetic energy, thus before writing the state I need to find that